In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2021-03"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:21]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '4120',
 '5330',
 '4279',
 '5520',
 '4759',
 '4190',
 '5830',
 '4419',
 '5570',
 '5490',
 '3510',
 '5170',
 '5590',
 '5081']

In [6]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + date + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "-" + date + "-data.parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "-" + date + "-data.parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)


<Response [200]>
5700


C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:383: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start instead
  'start': level._start,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:384: FutureWarning: RangeIndex._stop is deprecated and will be removed in a future version. Use RangeIndex.stop instead
  'stop': level._stop,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:385: FutureWarning: RangeIndex._step is deprecated and will be removed in a future version. Use RangeIndex.step instead
  'step': level._step


<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
4120
<Response [200]>
5330
<Response [200]>
4279
<Response [200]>
5520
<Response [200]>
4759
<Response [200]>
4190
<Response [200]>
5830
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5170
<Response [200]>
5590
<Response [200]>
5081
<Response [200]>


In [7]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data.parquet"

    foo = pq.read_table(file).to_pandas()
    
    recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top20-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [8]:
df.loc["CHINA","ALL PRODUCTS"]["2021"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2021-01-01,00,3.819920e+10,4.331589e+09,11.339476
2021-02-01,00,3.395794e+10,3.903253e+09,11.494374
2021-03-01,00,4.018032e+10,4.620955e+09,11.500542


In [26]:
date


'2020-11'

In [9]:
    file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

    foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,221313094,417,01,LIVE ANIMALS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,246092109,823,01,LIVE ANIMALS,2013-02,HS2
2,TOTAL FOR ALL COUNTRIES,261244392,75,01,LIVE ANIMALS,2013-03,HS2
3,TOTAL FOR ALL COUNTRIES,271133926,1389,01,LIVE ANIMALS,2013-04,HS2
4,TOTAL FOR ALL COUNTRIES,209785289,50,01,LIVE ANIMALS,2013-05,HS2
...,...,...,...,...,...,...,...
93,TOTAL FOR ALL COUNTRIES,484465278,9355681,93,ARMS AND AMMUNITION; PARTS AND ACCESSORIES THE...,2021-03,HS2
94,TOTAL FOR ALL COUNTRIES,6195877496,485481580,94,FURNITURE; BEDDING ETC; LAMPS NESOI ETC,2021-03,HS2
95,TOTAL FOR ALL COUNTRIES,475117816,12833069,91,CLOCKS AND WATCHES AND PARTS THEREOF,2021-03,HS2
96,TOTAL FOR ALL COUNTRIES,1836070435,4659583,88,"AIRCRAFT, SPACECRAFT, AND PARTS THEREOF",2021-03,HS2


In [28]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL,CTY_CODE
0,SINGAPORE,295720,0,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2020-11,HS2,5590
1,SINGAPORE,0,0,14,VEGETABLE PLAITING MATERIALS & PRODUCTS NESOI,2020-11,HS2,5590
2,SINGAPORE,3223114,554,15,"ANIMAL OR VEGETABLE FATS, OILS ETC. & WAXES",2020-11,HS2,5590
3,SINGAPORE,581755,0,16,"EDIBLE PREPARATIONS OF MEAT, FISH, CRUSTACEANS...",2020-11,HS2,5590
4,SINGAPORE,11400,23,17,SUGARS AND SUGAR CONFECTIONARY,2020-11,HS2,5590
...,...,...,...,...,...,...,...,...
80,SINGAPORE,200233,7351,44,WOOD AND ARTICLES OF WOOD; WOOD CHARCOAL,2020-11,HS2,5590
81,SINGAPORE,26432,588,62,"APPAREL ARTICLES AND ACCESSORIES, NOT KNIT ETC.",2020-11,HS2,5590
82,SINGAPORE,3839,0,72,IRON AND STEEL,2020-11,HS2,5590
83,SINGAPORE,2127459,39665,73,ARTICLES OF IRON OR STEEL,2020-11,HS2,5590
